# Food API

In [1]:
import requests

# Define your product search term
product_name = "raisin"
search_url = "https://world.openfoodfacts.org/cgi/search.pl"

# Parameters for the API request
params = {
    "search_terms": product_name,
    "search_simple": 1,
    "json": 1,
    "page_size": 5,  # Limit the number of results for performance
}

# Make the request
response = requests.get(search_url, params=params)

# Process the results
if response.status_code == 200:
    data = response.json()
    if data.get("products"):
        print(f"Found {len(data['products'])} products matching '{product_name}':\n")
        print("Full JSON response for all products:\n")
        # Print the full JSON response for all matching products
        for product in data["products"]:
            print(product)  # Print the full product JSON
    else:
        print("No products found for your search term.")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


Found 5 products matching 'raisin':

Full JSON response for all products:

{'_id': '3229820129488', '_keywords': ['ab', 'abricot', 'agriculture', 'ajoute', 'alemania', 'aliment', 'alimentaire', 'aux', 'azucar', 'bajo', 'base', 'bio', 'biologique', 'bjorg', 'boisson', 'cereale', 'certifiee', 'complete', 'de', 'de-oko-001', 'derive', 'ecocert', 'en', 'entreprise', 'et', 'europeen', 'europeenne', 'fibre', 'figue', 'fr-bio-01', 'fruit', 'gb-org-05', 'graine', 'hecho', 'melange', 'muesli', 'non', 'origine', 'petit-dejeuner', 'planet', 'point', 'pomme', 'pour', 'raisin', 'riche', 'san', 'score', 'sin', 'source', 'sucre', 'terre', 'ue', 'ue-non', 'union', 'vegetale', 'vegetaux', 'vert'], 'added_countries_tags': [], 'additives_n': 0, 'additives_original_tags': [], 'additives_tags': [], 'allergens': 'en:gluten,fr:Avoine', 'allergens_from_ingredients': 'en:gluten, en:gluten, en:gluten, en:gluten, blé, blé', 'allergens_from_user': '(fr) en:gluten,fr:Avoine', 'allergens_hierarchy': ['en:gluten', '

In [7]:
import requests

# Define your product search term
product_name = "Muesli Raisin, Figue, Abricot"
search_url = "https://world.openfoodfacts.org/cgi/search.pl"

# Parameters for the API request
params = {
    "search_terms": product_name,
    "search_simple": 1,
    "json": 1,
    "page_size": 1,  # Limit results for simplicity
}

# Make the request
response = requests.get(search_url, params=params)

# Process the results
if response.status_code == 200:
    data = response.json()
    if data.get("products"):
        product = data["products"][0]  # Take the first product
        product_name = product.get("product_name", "Unknown Product")
        barcode = product.get("code", "N/A")
        carbon_footprint_100g = product.get("carbon_footprint_100g", None)
        carbon_footprint_debug = product.get("carbon_footprint_from_known_ingredients_debug", None)
        agribalyse_co2_total = product.get("agribalyse", {}).get("co2_total", None)

        print(f"Product Name: {product_name}")
        print(f"Barcode: {barcode}")
        
        # Display carbon footprint information
        if carbon_footprint_100g:
            print(f"Carbon Footprint (per 100g): {carbon_footprint_100g} g CO₂e")
        elif agribalyse_co2_total:
            print(f"Agribalyse Total CO₂ Emissions: {agribalyse_co2_total} kg CO₂e")
        elif carbon_footprint_debug:
            print(f"Estimated Carbon Footprint (from ingredients): {carbon_footprint_debug}")
        else:
            print("Carbon Footprint data is not available.")
    else:
        print("No products found for your search term.")
else:
    print(f"Failed to retrieve data. HTTP Status code: {response.status_code}")


Product Name: Muesli Raisin, Figue, Abricot
Barcode: 3229820129488
Estimated Carbon Footprint (from ingredients): en:dried-apricots 5% x 1.1 = 5.5 g - 


In [4]:
!pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [6]:
# Load Agribalyse data
import pandas as pd

file_path = '../data/agribalyse-31-detail-par-ingredient.csv'
agribalyse_data = pd.read_csv(file_path)

# Search for a product by name or category
search_term = "Muesli"
matched_products = agribalyse_data[agribalyse_data["Nom Français"].str.contains(search_term, case=False, na=False)]

# If matches are found, calculate carbon footprint
if not matched_products.empty:
    print(f"Found {len(matched_products)} matches in Agribalyse for '{search_term}':")
    for _, product in matched_products.iterrows():
        product_name = product["Nom Français"]
        carbon_footprint_per_kg = product["Changement climatique"]
        weight_kg = 0.250  # Example: 250g
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- {product_name}: {carbon_footprint:.2f} kg CO₂e for 250g")
else:
    print(f"No matches found for '{search_term}' in Agribalyse.")


Found 56 matches in Agribalyse for 'Muesli':
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.02 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.15 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.11 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.13 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.01 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.02 kg CO₂e for 250g
- Muesli croustillant au chocolat (non enrichi en vitamines et minéraux): 0.20 kg CO₂e for 250g
- Muesli croustillant au chocolat, avec ou sans fruits, enrichi en vitamines et minéraux: 0.02 kg CO₂e for 250g
- Muesli croustillant au chocolat, avec ou sans fruits, enrichi en vitamines et minéraux: 0.15 kg CO₂e for 250g
- Muesli croustillant au chocolat, avec ou sans fruits, enr

In [30]:
!pip install unidecode
!pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)


In [48]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# Chemin vers le fichier Agribalyse
file_path = "../data/agribalyse-31-detail-par-ingredient.csv"
agribalyse_data = pd.read_csv(file_path)

def search_in_agribalyse_with_similarity(search_term, weight_kg):

    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

    # Combine French and English stopwords
    french_stopwords = set(stopwords.words("french")) | ENGLISH_STOP_WORDS
    
    # Normalize data
    agribalyse_data["Nom Français"] = (
        agribalyse_data["Nom Français"].str.strip().str.lower().apply(unidecode.unidecode)
    )
    search_term = unidecode.unidecode(search_term.strip().lower())

    # Prepare data
    product_names = agribalyse_data["Nom Français"].dropna().tolist()
    product_names.append(search_term)

    # Vectorize
    stop_words_list = list(french_stopwords)

    # Vectorize
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words=stop_words_list)
    tfidf_matrix = vectorizer.fit_transform(product_names)

    # Compute cosine similarity
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]
    best_match_product = agribalyse_data.iloc[best_match_index]

    # Debugging outputs
    print("Top 5 Similarity Scores and Matches:")
    top_indices = similarities.argsort()[-5:][::-1]
    for idx in top_indices:
        match_name = agribalyse_data.iloc[idx]["Nom Français"]
        score = similarities[idx]
        print(f" - {match_name}: {score:.2f}")

    if best_match_score > 0.15:  # Lower threshold
        product_name = best_match_product["Nom Français"]
        carbon_footprint_per_kg = best_match_product["Changement climatique"]
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- Closest match in Agribalyse: {product_name} (Similarity: {best_match_score:.2f})")
        print(f"  Carbon Footprint: {carbon_footprint:.2f} kg CO₂e for {weight_kg * 1000:.0f}g")
    else:
        print(f"No sufficiently similar products found for '{search_term}'.")



# Fonction pour chercher dans l'API Open Food Facts
def search_in_open_food_facts(product_name, weight_kg):
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)  # Timeout de 5 secondes
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]  # Prendre le premier produit
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)
                agribalyse_co2_total = product.get("agribalyse", {}).get("co2_total", None)
                
                print(f"Product Name: {product_name}")
                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    print(f"Carbon Footprint (per {weight_kg * 1000:.0f}g): {carbon_footprint:.2f} g CO₂e")
                elif agribalyse_co2_total:
                    carbon_footprint = float(agribalyse_co2_total) * weight_kg
                    print(f"Agribalyse Total CO₂ Emissions: {carbon_footprint:.2f} kg CO₂e")
                else:
                    print("Carbon Footprint data is not available in Open Food Facts.")
            else:
                print("No products found for your search term in Open Food Facts.")
                print("Switching to Agribalyse...")
                search_in_agribalyse_with_similarity(product_name, weight_kg)
        else:
            print(f"Failed to retrieve data from Open Food Facts. HTTP Status: {response.status_code}")
            print("Switching to Agribalyse...")
            search_in_agribalyse_with_similarity(product_name, weight_kg)
    except requests.exceptions.Timeout:
        print("Open Food Facts API timeout after 5 seconds. Switching to Agribalyse...")
        search_in_agribalyse_with_similarity(product_name, weight_kg)
    except Exception as e:
        print(f"An error occurred: {e}")
        search_in_agribalyse_with_similarity(product_name, weight_kg)

# Nom du produit et poids (exemple : 250g)
product_name = "muesli"
weight_kg = 0.250  # 250g

# Chercher dans Open Food Facts avec fallback sur Agribalyse
search_in_open_food_facts(product_name, weight_kg)


Product Name: cruesly mélange de noix
Carbon Footprint data is not available in Open Food Facts.


In [57]:
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
from nltk.corpus import stopwords

# Load Agribalyse data
file_path = "../data/agribalyse-31-detail-par-ingredient.csv"
agribalyse_data = pd.read_csv(file_path)

def normalize_text(text):
    """Normalize text by removing accents and converting to lowercase."""
    return unidecode.unidecode(text.strip().lower())

def filter_meaningful_components(components):
    """Filter out stopwords and retain meaningful components."""
    french_stopwords = {"de", "et", "aux", "pour", "avec", "sur", "au"}  # Add more as needed
    return [component for component in components if component not in french_stopwords]

def search_in_agribalyse_with_similarity(search_term, weight_kg):
    """Search in Agribalyse with cosine similarity."""
    # Normalize data
    agribalyse_data["Nom Français"] = (
        agribalyse_data["Nom Français"].str.strip().str.lower().apply(unidecode.unidecode)
    )
    search_term = normalize_text(search_term)

    # Prepare data
    product_names = agribalyse_data["Nom Français"].dropna().tolist()
    product_names.append(search_term)

    # Vectorize
    vectorizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(product_names)

    # Compute cosine similarity
    similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
    best_match_index = similarities.argmax()
    best_match_score = similarities[best_match_index]
    best_match_product = agribalyse_data.iloc[best_match_index]

    # Debugging outputs
    print("Top 5 Similarity Scores and Matches:")
    top_indices = similarities.argsort()[-5:][::-1]
    for idx in top_indices:
        match_name = agribalyse_data.iloc[idx]["Nom Français"]
        score = similarities[idx]
        print(f" - {match_name}: {score:.2f}")

    if best_match_score > 0.2:  # Threshold for similarity
        product_name = best_match_product["Nom Français"]
        carbon_footprint_per_kg = best_match_product["Changement climatique"]
        carbon_footprint = carbon_footprint_per_kg * weight_kg
        print(f"- Closest match in Agribalyse: {product_name} (Similarity: {best_match_score:.2f})")
        print(f"  Carbon Footprint: {carbon_footprint:.2f} kg CO₂e for {weight_kg * 1000:.0f}g")
    else:
        print(f"No sufficiently similar products found for '{search_term}'.")

def search_in_agribalyse_with_components(search_term, weight_kg):
    """Break down search term into components and search in Agribalyse."""
    components = filter_meaningful_components(search_term.split())
    for component in components:
        print(f"Searching Agribalyse for component: '{component}'...")
        search_in_agribalyse_with_similarity(component, weight_kg)

def search_in_open_food_facts(product_name, weight_kg):
    """Search in Open Food Facts and fallback to Agribalyse if needed."""
    search_url = "https://world.openfoodfacts.org/cgi/search.pl"
    params = {
        "search_terms": product_name,
        "search_simple": 1,
        "json": 1,
        "page_size": 1,
    }

    try:
        response = requests.get(search_url, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            if data.get("products"):
                product = data["products"][0]
                product_name = product.get("product_name", "Unknown Product")
                carbon_footprint_100g = product.get("carbon_footprint_100g", None)
                
                print(f"Product Name: {product_name}")
                if carbon_footprint_100g:
                    carbon_footprint = (float(carbon_footprint_100g) / 100) * weight_kg * 1000
                    print(f"Carbon Footprint (per {weight_kg * 1000:.0f}g): {carbon_footprint:.2f} g CO₂e")
                else:
                    print("Carbon Footprint data is not available in Open Food Facts.")
                    print("Switching to Agribalyse with components...")
                    search_in_agribalyse_with_components(product_name, weight_kg)
            else:
                print("No products found for your search term in Open Food Facts.")
                print("Switching to Agribalyse...")
                search_in_agribalyse_with_similarity(product_name, weight_kg)
        else:
            print(f"Failed to retrieve data from Open Food Facts. HTTP Status: {response.status_code}")
            print("Switching to Agribalyse...")
            search_in_agribalyse_with_similarity(product_name, weight_kg)
    except requests.exceptions.Timeout:
        print("Open Food Facts API timeout after 5 seconds. Switching to Agribalyse...")
        search_in_agribalyse_with_similarity(product_name, weight_kg)
    except Exception as e:
        print(f"An error occurred: {e}")
        search_in_agribalyse_with_similarity(product_name, weight_kg)

# Test Input
product_name = "soda à l'orange"
weight_kg = 1  # 250g
search_in_open_food_facts(product_name, weight_kg)


Open Food Facts API timeout after 5 seconds. Switching to Agribalyse...
Top 5 Similarity Scores and Matches:
 - nectar d'orange: 0.22
 - nectar d'orange: 0.22
 - nectar d'orange: 0.22
 - marmelade d'orange: 0.21
 - marmelade d'orange: 0.21
- Closest match in Agribalyse: nectar d'orange (Similarity: 0.22)
  Carbon Footprint: 0.00 kg CO₂e for 250g
